# Tutorial 3: Retrieval

In diesem dritten Tutorial wollen wir erste Rankings erzeugen und diese anschließend auswerten.  

Hierzu verwenden wir weiterhin PyTerrier und die LongEval Testkollektion, die im ersten Tutorial indexiert und im zweiten Tutorial analysiert wurde.

Bevor wir PyTerrier benutzen können müssen wir es importieren und initialisieren.

In [1]:
import pandas as pd
import pyterrier as pt
pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
/var/folders/pz/f66p4s0s11z2c1c1l3v0qbh40000gn/T/ipykernel_95658/1997712876.py:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


Anschließend laden wir den Index.

In [2]:
index = pt.IndexFactory.of("./index")
print(index.getCollectionStatistics().toString())

Number of documents: 1570734
Number of terms: 3694081
Number of postings: 433928454
Number of fields: 0
Number of tokens: 777790536
Field names: []
Positions:   true



Als nächstes definieren wir die zwei Suchsysteme Tf und BM25.

In [3]:
tf = pt.BatchRetrieve(index, wmodel="Tf")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

/var/folders/pz/f66p4s0s11z2c1c1l3v0qbh40000gn/T/ipykernel_95658/77646355.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tf = pt.BatchRetrieve(index, wmodel="Tf")
/var/folders/pz/f66p4s0s11z2c1c1l3v0qbh40000gn/T/ipykernel_95658/77646355.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")


Diese Systeme können wir nun verwenden, um die Testkollektion zu durchsuchen. Hierzu verwenden wir die `search` Funktion. Als Ergebnis bekommen wir einen Dataframe mit den gerankten Dokumenten zurück.

In [4]:
tf.search("Hello World")

,qid,docid,docno,rank,score,query
0,1,1336822,doc062208001049,0,100.0,Hello World
1,1,1538955,doc062208607798,1,95.0,Hello World
2,1,32434,doc062214401498,2,94.0,Hello World
3,1,41467,doc062214410531,3,93.0,Hello World
4,1,31039,doc062214400103,4,91.0,Hello World
...,...,...,...,...,...,...
995,1,1452339,doc062205207149,995,20.0,Hello World
996,1,1464932,doc062204401620,996,20.0,Hello World
997,1,1468000,doc062204404688,997,20.0,Hello World
998,1,1471329,doc062204408017,998,20.0,Hello World


In [5]:
# TODO:
# Durchsuche die Testkollektion mit verschiedenen Querys.

Um die zwei Systeme zu vergleichen, müssen wir ein Experiment durchführen. Praktischerweise hilft PyTerrier hierbei. Die `experiment` Funktion tut genau das. Als Input übergeben wir:

- `retr_systems`: Eine Liste Retrieval Systeme die wir vergleichen wollen.
- `topics`: Die Anfragen die verwendet werden, um die Sucheche durchzuführen.
- `qrel`: Die Relevanzbewertungen aus der Testkollektion.
- `eval_metrics`: Eine Liste Retrieval Maße die verwendet werden, um die Effektivität zu bemessen.

Eine Übersichtsliste der unterstützten Maße ist in der [Dokumentation](https://pyterrier.readthedocs.io/en/latest/experiments.html#available-evaluation-measures) zu finden.

In [21]:
topics = pd.read_csv("../data/raw/longeval-small/LongEval/2023_train/publish/English/Queries/train.tsv",
            sep="\t",
            names=["qid", "query"],
        )
topics["query"] = topics["query"].str.replace("/", "")
topics["query"] = topics["query"].str.replace("'", "")

In [22]:
qrels = pd.read_csv("../data/raw/longeval-small/LongEval/2023_train/publish/French/Qrels/train.txt",
            sep=" ",
            names=["qid", "Q0", "docno", "relevance"]
)

In [ ]:
results = pt.Experiment(
    retr_systems=[tf, bm25],
    topics=topics,
    qrels=qrels,
    eval_metrics=["ndcg"],
    verbose = True
)
results

In [ ]:
# TODO:
# Führe Experimente durch und vergleiche die Ergebnisse anhand der Maße
# Kannst du die bisher besprochenen Eigenschaften der unterschiedlichen
# Retrieval Maße beobachten?

Um einen Detaillierteren Einblick in die Ergebnisse zu bekommen, lassen wir uns nun die Ergebnisse pro Topic, also pro Suchanfrage zurückgeben. Diese Ergebnisse lassen weitere Analysen zu. Interessante Fragestellungen sind zum Beispiel:
- Für welches Topic ist ein System am effektivsten und für welches am wenigsten effektiv?
- Woran könnte das liegen?
- Gibt es Bereiche in denen ein System besonders gut performt?
- Wie lassen sich die Ergebnisse visualisieren?

In [27]:
results = pt.Experiment(
    retr_systems=[tf, bm25],
    topics=topics,
    qrels=qrels,
    perquery=True,
    eval_metrics=["ndcg_cut_10"]
)
results

,name,qid,measure,value
980,TerrierRetr(BM25),q062210081,ndcg_cut_10,0.000000
982,TerrierRetr(BM25),q062210089,ndcg_cut_10,0.000000
983,TerrierRetr(BM25),q062210095,ndcg_cut_10,0.260235
984,TerrierRetr(BM25),q062210106,ndcg_cut_10,0.000000
985,TerrierRetr(BM25),q062210117,ndcg_cut_10,0.301030
...,...,...,...,...
303,TerrierRetr(Tf),q06229696,ndcg_cut_10,0.000000
304,TerrierRetr(Tf),q06229698,ndcg_cut_10,0.000000
37,TerrierRetr(Tf),q0622984,ndcg_cut_10,0.000000
305,TerrierRetr(Tf),q06229906,ndcg_cut_10,0.000000


In [ ]:
# TODO
# Analysiere die Ergebnisse pro Topic anhand der oben genannten und eigenen
# Fragestellungen.